# Chuẩn bị dữ liệu

In [8]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings

# Loại bỏ những thông báo không cần thiết trong quá trình thực thi
warnings.filterwarnings('ignore')

# Load bộ dữ liệu vào biến 'data'
file_path = './social_media_data_1000 (1).csv'
data = pd.read_csv(file_path)

# Một số quan sát về dữ liệu